In [1]:
# import all necessary libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten,GlobalAveragePooling2D
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [ ]:
#Data directory path for display or show the image from the directory

data_dir = ('image directiory or path') # traininvg or validation image path
categories = ['NT', 'SBT', 'SMT', 'TBT', 'TMT', 'SBSMT']# Put your class names
 n_classes = len(categories)
for i in categories:
    path = os.path.join(data_dir, i)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img))


# Image Visualize
plt.imshow(img_array);
# The image shape.
img_array.shape

# Define Image shape
img_height = 299
img_width = 299
n_classes = 6 # Number of class for

In [ ]:
# Image Data generator for training and Validation

image_generator = ImageDataGenerator(rescale=1./255,
                                    featurewise_center=False,
                                    samplewise_center=False,
                                    featurewise_std_normalization=False,
                                    samplewise_std_normalization=False,
                                    zca_whitening=False,
                                    rotation_range=0,
                                    zoom_range = 0,
                                    width_shift_range=0,
                                    height_shift_range=0,
                                    horizontal_flip=True,
                                    vertical_flip=False,
                                    validation_split=0.0)

test_generator = ImageDataGenerator(rescale=1./255,
                                    validation_split=0.0)

train = image_generator.flow_from_directory('training image path', target_size=(img_height, img_width),
                                              batch_size=16, class_mode= "categorical", color_mode='rgb', subset='training', interpolation='bicubic') # This is only when you have a separate directory for 80% training image

valid = test_generator.flow_from_directory('Validation Image path', target_size=(img_height, img_width),
                                              batch_size=16, class_mode= "categorical", color_mode='rgb', subset='training', interpolation='bicubic') # This is only when you have a separate directory for 20% validation image

# you can also use the following codes for if you have all image dataset in a directory and the code automatically split the whole dataset into training and validation

#dir_path="image dataset directory path"
#train_data = image_generator.flow_from_directory(dir_path, target_size=(img_height, img_width), batch_size=16, validation_split=0.2,class_mode= "categorical", color_mode='rgb', subset='training', interpolation='bicubic') # Split the dataset into 80% training dataset

#valid_data = test_generator.flow_from_directory(dir_path, target_size=(img_height, img_width), batch_size=16, validation_split=0.2, class_mode= "categorical", color_mode='rgb', subset='validation', interpolation='bicubic') # Split the dataset into 20% validation dataset

# Please use the above varibales in the folowing line of codes when you need #


In [ ]:
# Plotting Confusion Matrix

import itertools
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):

    #This function prints and plots the confusion matrix. Normalization can be applied by setting `normalize=True`.

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], '.2f'),
            horizontalalignment="center",
            color="black" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')

In [ ]:
# Plotting Accuracy and Loss Curve

import matplotlib.pyplot as plt
#plot loss and accuracy helper function
def plot_accuracy(history):
    plt.plot(history.history['categorical_accuracy'])
    plt.plot(history.history['val_categorical_accuracy'])
    #plt.title('model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Training Accuracy', 'Validation Accuracy'], loc='lower right')
    plt.show()

def plot_loss(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    #plt.title('model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Training Loss', 'Validation Loss'], loc='upper right')
    plt.show()

In [ ]:
# Proposed FT-DTL Model creation

Base_model= tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# The last 15 layers fine tune
for layer in Base_model.layers:
    layer.trainable = False

# Fine-tuned additional layers to base model
x = Base_model.output
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(0.3)(x)
output  = Dense(units=n_classes, activation='softmax')(x)
proposed_FT_FEDTL = Model(Base_model.input, output)
proposed_FT_FEDTL.compile(optimizer="adam",loss='categorical_crossentropy',metrics=['categorical_accuracy'])
#proposed_FT_FEDTL.summary() # optional

In [ ]:
# Model compile
proposed_FT_FEDTL.compile(optimizer="adam",loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [ ]:
# Set up fine-tuned hyperparameters and start the training of the model

model_es = EarlyStopping(monitor = 'loss', min_delta = 1e-11, patience = 40, verbose = 1)
model_rlr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 40, verbose = 1)
model_mcp = ModelCheckpoint(filepath = 'inp_model_weights.h5', monitor = 'val_categorical_accuracy',
                      save_best_only = True, verbose = 1)
history = proposed_FT_FEDTL.fit(train, steps_per_epoch=1050//16, epochs=100, validation_data=valid, validation_steps= 200//16, callbacks=[model_es, model_rlr, model_mcp])

In [ ]:
# Call Accuracy and Loss functions to print the graphs

plot_accuracy(history)
plot_loss(history)

In [ ]:
# Testing the samples prediction

test_samples = test_generator.flow_from_directory('testing image path', target_size=(img_height, img_width),
                                              batch_size=16, class_mode= "categorical", color_mode='rgb', interpolation='bicubic')
test_imgs, test_labels = next(test_samples)
rounded_labels = np.argmax(test_labels, axis=-1)
predictions = proposed_FT_FEDTL.predict_generator(test_samples, steps=1, verbose=0)
rounded_prediction = np.argmax(predictions, axis=-1)

In [ ]:
# Call Confusion matrix function to plot the matrix
cm = confusion_matrix(y_true=rounded_labels, y_pred=rounded_prediction)
# plot confusion matrix
plot_confusion_matrix(cm=cm, classes = categories, title='confusion_matrix')

In [ ]:
# Print Classification  report

from sklearn.metrics import classification_report
print(classification_report(rounded_labels,rounded_prediction))

In [ ]:
# Proposed Model Evaluate

for imgbatch, labelbatch in test_samples:
  print("Batch Size: ", imgbatch.shape)
  print("Label Batch Size: ", labelbatch.shape)

test_loss, test_accuracy=proposed_FT_FEDTL.evaluate(test_samples)
print("Test Accuracy:", test_accuracy)
print("Test Loss:", test_loss)

In [ ]:
# image sample testing result
image=cv2.imread(str(result[0]))
image_resize=cv2.resize(image,(img_height,img_width ))
image=np.expand_dims(image_resize, axis=0)
print(image.shape)
prediction=proposed_FT_FEDTL.predict(image)
print(prediction)
outputclass=categories[np.argmax(prediction)]
print("The predicted class Name is :", outputclass)